In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

## 0. DataFrame

In [ ]:
football_df = pd.read_csv('all_data_with_elo.csv', low_memory = False)
football_df

## 1. Descriptive Statistics 

**1.1 DataFrame Shape**

In [ ]:
# no. rows and no. cols
football_df.shape

In [ ]:
# feature names
print(football_df.columns.tolist())

**1.2 NaN Values**

In [ ]:
football_df.isnull().sum()

In [ ]:
# total elements in 
football_df.size

In [ ]:
# total number of NaN
football_df.size - football_df.count().sum()

In [ ]:
# total number of NaN rows
football_df.isnull().any(axis = 1).sum()

In [ ]:
# total number of NaN columns
football_df.isnull().any(axis = 0).sum()

## 2. Data Wrangling and Feature Transformation/Development

### 2.1 NaN Handling

`TODO`: drop NaN values along columns: {Date, Home Team, Away Team, FTR} <br>
`TODO`: identify betting odds w/ most available data

In [ ]:
nan_mask = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A', 
            'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'HomeTeamELO', 'AwayTeamELO']

In [ ]:
#football_df.FTR.replace('nan', np.nan, inplace=True)
nan_football_df = football_df.dropna(subset = nan_mask)
nan_football_df

In [ ]:
# resize shape
football_df.shape[0] - nan_football_df.shape[0]

### 2.2 Feature Encoding <br>
* $\phi(Date)$ $\Rightarrow$ one column for *year*, second column for *month*, third column for *day of year*
* One hot encode Division, Home and Away Teams
* Label encode Full Time Result (Win/Draw/Loss)

In [ ]:
feats = nan_mask

In [ ]:
learning_df = nan_football_df.copy()[feats]
learning_df.reset_index(inplace=True, drop=True)

**2.2.1 Division and Home/Away Team Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()

In [ ]:
onehot_div = div_encoder.fit_transform(learning_df.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(learning_df.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(learning_df.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

In [ ]:
learning_df = pd.concat([learning_df, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
learning_df.drop(columns = ['Div'], inplace = True)

**2.2.2 Full Time Result Encoding**

In [ ]:
target_encoder = LabelEncoder()
learning_df['Result'] = target_encoder.fit_transform(learning_df.FTR) 

**2.2.3 Date Encoding**

In [ ]:
learning_df['Year'] = pd.DatetimeIndex(learning_df.Date).year

learning_df['Month'] = pd.DatetimeIndex(learning_df.Date).month
learning_df['Sin_Month'] = np.sin(2*np.pi*learning_df.Month/12)
learning_df['Cos_Month'] = np.cos(2*np.pi*learning_df.Month/12)

learning_df['DayofYear'] = pd.DatetimeIndex(learning_df.Date).dayofyear
learning_df['Sin_Day'] = np.sin(2*np.pi*learning_df.DayofYear/365)
learning_df['Cos_Day'] = np.cos(2*np.pi*learning_df.DayofYear/365)

learning_df.drop(columns = ['Date','Month'], inplace = True)
# learning_df.drop(columns = ['Date'], inplace = True)

In [ ]:
learning_df

### 2.3 Feature Engineering <br>
* $\phi(x)$ feature transformation $\Rightarrow$ last match result, win/loss streak to date, wins to season date
* $\phi(x)$ feature engineering $\Rightarrow$ average the home, away, and draw odds

**2.3.1 Last Match Result** <br>
Indicate the result from the last match played between both teams

In [ ]:
def compute_lastmatchres(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['FTR'], inplace = True)
    return df

In [ ]:
learning_df = compute_lastmatchres(learning_df)
# learning_df.drop(columns = ['FTR'], inplace = True)

**2.3.2 Home and Away Win/Loss Streak** <br>
Important note about this feature: the win/loss streak is the teams *home* and *away* win streak, *not* its ***consecutive*** win/loss streak.

In [ ]:
# https://stackoverflow.com/questions/52976336/compute-winning-streak-with-pandas
# https://joshdevlin.com/blog/calculate-streaks-in-pandas/

In [ ]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])

In [ ]:
learning_df = compute_winstreak(learning_df)

**2.3.4 Season Home/Away Wins to Date** <br>
Indicate the number of wins for a team as home and away to date within current season

In [ ]:
toy = learning_df[(learning_df.Year == 2010) & (learning_df.HomeTeam == 'Barcelona')][['HomeTeam', 'AwayTeam', 'Result']]
toy['HomeWin'] = toy.Result.replace([0, 1, 2], [0, 0, 1])
toy['HomeWinsToDate'] = toy.HomeWin.cumsum()

In [ ]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])

In [ ]:
learning_df = compute_winstodate(learning_df)
learning_df.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)

**2.3.5 Website Odds** <br>
The `betting odds` recorded by various betting websites offer insight into sentiment surrounding the outcome of a particular game. 

In [ ]:
betting_feats = ['B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']
betting_feats

In [ ]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]

In [ ]:
learning_df = compute_meanodds(learning_df, betting_feats)

### 2.4 Peek @ Learning DataFrame

In [ ]:
learning_df

# 3. Model Development

* Establish a baseline Logistic Regression model fit over the entire learning dataframe without special regard to *division* and *team*. 
* Train model over 16 seasons, and predict for the remaining 3 seasons (approximate 80-20 split)

### 3.1 Train and Test Split

In [ ]:
split = 0.80
no_seasons = 20

print('No. seasons to train over: ' + str(round(split*no_seasons)))

In [ ]:
X, y = learning_df.loc[:, learning_df.columns != 'Result'], learning_df[['Result']]

In [ ]:
# full_feat = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'Last Match Result',
#              'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats

# exclude_feats = ['HomeWinsToDate', 'AwayWinsToDate', 'Last Match Result'] 

In [ ]:
# X = X[X.columns[~X.columns.isin(exclude_feats)]]
# X

In [ ]:
X

In [ ]:
xTr, xTe = X[X.Year <= 2016], X[X.Year > 2016]
yTr, yTe = y.loc[xTr.index, :], y.loc[xTe.index, :]

### 3.2 Normalization <br>
Following our various feature transformations and development, we arrived to a sparse dataframe with the exception of a few features(*Year, DayofYear*). It will be important to *normalize* these features as they are in gross magnitudes compared to the remaining features. During model training, having dominating features (in scale relative to others) can be dangerous as the weight updates may mistakengly favor these larger-scale features because it will have the largest influence on the target output. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
xTr.loc[:, ['Year']] = minmax_scaler.fit_transform(xTr.loc[:, ['Year']])
xTe.loc[:, ['Year']] = minmax_scaler.transform(xTe.loc[:, ['Year']])

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats

xTr.loc[:, to_scale] = std_scaler.fit_transform(xTr.loc[:, to_scale])
xTe.loc[:, to_scale] = std_scaler.transform(xTe.loc[:, to_scale])

In [ ]:
xTr

In [ ]:
xTe

### 3.3 HomeWins Baseline Model

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# training score
baseline_Tr = np.full((24233, 1), 2) 
accuracy_score(yTr.Result.values, baseline_Tr.ravel())

In [ ]:
# testing score
baseline_preds_Te = np.full((8628  , 1), 2) #predicts home wins all the time
accuracy_score(yTe.Result.values, baseline_preds_Te.ravel())

### 3.4 Multinomial Logistic Regression

**3.4.1** $l2$ Regularized

In [ ]:
from sklearn.linear_model import LogisticRegression
l2_lr = LogisticRegression(max_iter = 10000, n_jobs=-1).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, l2_lr.predict(xTr))

In [ ]:
# testing score
lr_preds = l2_lr.predict(xTe)
accuracy_score(yTe.Result.values, lr_preds)

**3.4.1** $l2$ Penalty Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

logistic_params = {'C':[0.001,0.01,0.10]}

logistic_randsearch = RandomizedSearchCV(estimator=LogisticRegression(max_iter=10000),
                                         param_distributions=logistic_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         cv=5,
                                         n_jobs=-1)

logistic_rand_results = logistic_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (logistic_rand_results.best_score_, logistic_rand_results.best_params_))

In [ ]:
l2_rs = logistic_rand_results.best_estimator_

In [ ]:
# training score
accuracy_score(yTr.Result.values, l2_rs.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, l2_rs.predict(xTe))

**3.4.4** $l1$ Regularized

In [ ]:
l1_lr = LogisticRegression(penalty='l1', solver='saga', max_iter = 10000, n_jobs=-1).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, l1_lr.predict(xTr))

In [ ]:
# testing score
l1_preds = l1_lr.predict(xTe)
accuracy_score(yTe.Result.values, l1_preds)

**3.4.5** Penalty Tuning

In [ ]:
l1_params = {'C':[0.001,0.01,0.10]}

l1_randsearch = RandomizedSearchCV(estimator=LogisticRegression(penalty='l1',solver='saga', max_iter=10000),
                                         param_distributions=l1_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         n_jobs=-1,
                                         cv=5)

l1_rand_results = l1_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (l1_rand_results.best_score_, l1_rand_results.best_params_))

In [ ]:
l1_rs = l1_randsearch.best_estimator_ #LogisticRegression(C=0.10, solver='saga', max_iter=10000).fit(xTr, yTr.values.ravel())#

In [ ]:
# training score
accuracy_score(yTr.Result.values, l1_rs.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, l1_rs.predict(xTe))

### 3.5 Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm = SVC(max_iter=100000).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, svm.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, svm.predict(xTe))

**3.5.2** Penalty Tuning

In [ ]:
svm_params = {'C':[0.001,0.01,0.10]}

svm_randsearch = RandomizedSearchCV(estimator=SVC(max_iter=100000),
                                         param_distributions=svm_params,
                                         scoring='accuracy',
                                         verbose=2,
                                         cv=5,
                                         n_jobs=-1)

svm_rand_results = svm_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (svm_rand_results.best_score_, svm_rand_results.best_params_))

In [ ]:
svm_rs = svm_rand_results.best_estimator_

In [ ]:
# training score
accuracy_score(yTr.Result.values, svm_rs.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, svm_rs.predict(xTe))

### 3.6 Simple Neural Network ####

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(512,),
                    activation='relu',
                    batch_size=64,
                    max_iter=1000,
                   ).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, mlp.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, mlp.predict(xTe))

### 3.7 Stacked Classifier ###

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
from sklearn.ensemble import StackingClassifier
stacked_clf = StackingClassifier(estimators=[('svm', SVC(max_iter=100000)), ('logistic', LogisticRegression(C=0.01, max_iter=10000))],
                                final_estimator=LogisticRegression(max_iter=10000),
                                n_jobs=-1).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, stacked_clf.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, stacked_clf.predict(xTe))

## 4. Result Analysis ##

In [ ]:
## TODO: breakdown results across divisions and/or teams; i.e., see how model performs individually at subgroups

## 5. Scrap Code ##

In [ ]:
barcelona_df = learning_df[(learning_df['HomeTeam 17'] == 1) | (learning_df['AwayTeam 17'] == 1)]
barcelona_df

In [ ]:
bxTr = xTr[(xTr['HomeTeam 17'] == 1) | (xTr['AwayTeam 17'] == 1)]
bxTe = xTe[(xTe['HomeTeam 17'] == 1) | (xTe['AwayTeam 17'] == 1)]

In [ ]:
byTr, byTe = yTr.loc[bxTr.index,:], yTe.loc[bxTe.index,:]

In [ ]:
# training score
accuracy_score(byTr, l1_lr.predict(bxTr))

In [ ]:
# testing score
accuracy_score(byTe, l1_lr.predict(bxTe))

In [ ]:
# training score
accuracy_score(byTr, l2_lr.predict(bxTr))

In [ ]:
# testing score
accuracy_score(byTe, l2_lr.predict(bxTe))